## 1.7 Solving the N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.


In [3]:
from copy import deepcopy

# definition of the problem
class NPuzzleState:

    def __init__(self, board, move_history=[]):
        # board(list[list[int]]) - the state of the board
        # move_history(list[list[list[int]]]) - the history of the moves up until this state
        self.board = deepcopy(board)
        (self.blank_row, self.blank_col) = self.find_blank()

        # create an empty array and append move_history
        self.move_history = [] + move_history + [self.board]

    def children(self):
        # returns the possible moves
        functions = [self.up, self.down, self.left, self.right]

        children = []
        for func in functions:
            child = func()
            if child:
                children.append(child)

        return children

    def find_blank(self):
        # finds the blank row and col
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] == 0:
                    return (row, col)

    def move(func):
        # decorator function to add to history everytime a move is made
        # functions with @move will apply this decorator
        def wrapper(self):
            state = NPuzzleState(self.board, self.move_history)
            value = func(state)
            if value:
                return state
            else:
                return None

        return wrapper

    @move
    def up(self):
        # moves the blank upwards
        if self.blank_row == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row - 1][self.blank_col]
            self.board[self.blank_row - 1][self.blank_col] = 0
            self.blank_row -= 1
            return True

    @move
    def down(self):
        # moves the blank downwards
        if self.blank_row == len(self.board) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row + 1][self.blank_col]
            self.board[self.blank_row + 1][self.blank_col] = 0
            self.blank_row += 1
            return True

    @move
    def left(self):
        # moves the blank left
        if self.blank_col == 0:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col - 1]
            self.board[self.blank_row][self.blank_col - 1] = 0
            self.blank_col -= 1
            return True

    @move
    def right(self):
        # moves the blank right
        if self.blank_col == len(self.board[0]) - 1:
            return False
        else:
            self.board[self.blank_row][self.blank_col] = self.board[self.blank_row][self.blank_col + 1]
            self.board[self.blank_row][self.blank_col + 1] = 0
            self.blank_col += 1
            return True

    def is_complete(self):
        # checks if the board is complete
        for row in range(len(self.board)):
            for col in range(len(self.board[0])):
                if self.board[row][col] != row * len(self.board[0]) + col + 1 and self.board[row][col] != 0:
                    return False
        return True

    def __hash__(self):
        # to be able to use the state in a set
        return hash(str([item for sublist in self.board for item in sublist]))

    def __eq__(self, other):
        # compares the two matrices
        return [item for sublist in self.board for item in sublist] == [item for sublist in other.board for item in sublist]

def print_sequence(sequence):
    print("Steps:", len(sequence) - 1)
    # prints the sequence of states
    for state in sequence:
        for row in state:
            print(row)
        print()


def problems():
    return (
        NPuzzleState([[1, 2, 3], [5, 0, 6], [4, 7, 8]]),
        NPuzzleState([[1, 3, 6], [5, 2, 0], [4, 7, 8]]),
        NPuzzleState([[1, 6, 2], [5, 7, 3], [0, 4, 8]]),
        NPuzzleState([[5, 1, 3, 4], [2, 0, 7, 8], [10, 6, 11, 12], [9, 13, 14, 15]]),
    )

**b)** Implement code to solve this problem using the “breadth-first” strategy (in this case
identical to "Uniform Cost"). 

In [4]:
def bfs(problem):
    queue = [problem]  # Queue for BFS
    visited = set()  # Set to keep track of visited states

    while queue:
        current_state = queue.pop(0)  # Dequeue the next state

        if current_state.is_complete():  # Check if the current state is the goal state
            return current_state.move_history  # Return the move history to the goal state

        visited.add(hash(current_state))  # Mark the current state as visited

        for child in current_state.children():  # Explore children
            if hash(child) not in visited:  # Only consider child if it hasn't been visited
                queue.append(child)  # Enqueue child
                visited.add(hash(child))  # Mark child as visited to avoid revisiting

    return None  # Return None if no solution is found

# prints the sequence for the first problem using bfs
print_sequence(bfs(problems()[2]))

Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



**c)** Implement code to solve this problem using Greedy Search and using the A*
Algorithm.

Suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

In [5]:
import heapq # we'll be using a heap to store the states

def greedy_search(problem, heuristic):
    setattr(NPuzzleState, "__lt__", lambda self, other: heuristic(self) < heuristic(other))
    states = [(heuristic(problem), problem)]
    visited = set()
    
    while states:
        _, current_state = heapq.heappop(states)
        if current_state.is_complete():
            return current_state.move_history
        visited.add(current_state)
        for child in current_state.children():
            if child not in visited:
                heapq.heappush(states, (heuristic(child), child))
    return None


def _preferential_position(number, side):
    # calculates the preferred position of a piece given its number
    # number (int) - the number of the piece
    # side (int) - the size of the side of the board (only for square boards)
    if number == 0:
        # if it is the last piece, it is 0
        row = col = side - 1
    else:
        # otherwise it is sequential, starting at 1
        row = number // side
        col = number % side - 1
    return (row, col)

def h1(state):
    # heuristic function 1
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)
    total = 0
    for row in range(side):
        for col in range(side):
            if board[row][col] != 0 and board[row][col] != row * side + col + 1:
                total += 1
    return total


def h2(state):
    # heuristic function 2
    board = state.board
    side = len(board) # the size of the side of the board (only for square boards)
    total = 0
    for row in range(side):
        for col in range(side):
            number = board[row][col]
            if number != 0:
                goal_row, goal_col = _preferential_position(number, side)
                total += abs(goal_row - row) + abs(goal_col - col)
    return total


print('h1')
print_sequence(greedy_search(problems()[2], h1))

print('h2')
print_sequence(greedy_search(problems()[2], h2))


h1
Steps: 48
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 7, 3]
[4, 8, 0]

[1, 6, 2]
[5, 7, 0]
[4, 8, 3]

[1, 6, 2]
[5, 0, 7]
[4, 8, 3]

[1, 6, 2]
[0, 5, 7]
[4, 8, 3]

[1, 6, 2]
[4, 5, 7]
[0, 8, 3]

[1, 6, 2]
[4, 5, 7]
[8, 0, 3]

[1, 6, 2]
[4, 5, 7]
[8, 3, 0]

[1, 6, 2]
[4, 5, 0]
[8, 3, 7]

[1, 6, 2]
[4, 0, 5]
[8, 3, 7]

[1, 0, 2]
[4, 6, 5]
[8, 3, 7]

[1, 2, 0]
[4, 6, 5]
[8, 3, 7]

[1, 2, 5]
[4, 6, 0]
[8, 3, 7]

[1, 2, 5]
[4, 0, 6]
[8, 3, 7]

[1, 2, 5]
[4, 3, 6]
[8, 0, 7]

[1, 2, 5]
[4, 3, 6]
[8, 7, 0]

[1, 2, 5]
[4, 3, 0]
[8, 7, 6]

[1, 2, 5]
[4, 0, 3]
[8, 7, 6]

[1, 0, 5]
[4, 2, 3]
[8, 7, 6]

[1, 5, 0]
[4, 2, 3]
[8, 7, 6]

[1, 5, 3]
[4, 2, 0]
[8, 7, 6]

[1, 5, 3]
[4, 0, 2]
[8, 7, 6]

[1, 5, 3]
[4, 7, 2]
[8, 0, 6]

[1, 5, 3]
[4, 7, 2]
[0, 8, 6]

[1, 5, 3]
[0, 7, 2]
[4, 8, 6]

[1, 5, 3]
[7, 0, 2]
[4, 8, 6]

[1, 5, 3]
[7, 2, 0]
[4, 8, 6]

[1, 5, 3]
[7, 2, 6]
[4, 8, 0]

[1, 5, 3]
[7, 2, 6]
[4, 0, 8]

[1, 5, 3]
[7, 2, 6]
[0, 4, 8]

[1, 5, 3]
[0, 2, 6]
[7, 4,

In [6]:
def a_star_search(problem, heuristic):
    setattr(NPuzzleState, "__lt__", lambda self, other: len(self.move_history) + heuristic(self) < len(other.move_history) + heuristic(other))
    states = [(heuristic(problem), problem)]
    visited = set()
    
    while states:
        _, current_state = heapq.heappop(states)
        if current_state.is_complete():
            return current_state.move_history
        visited.add(current_state)
        for child in current_state.children():
            if child not in visited:
                heapq.heappush(states, (len(child.move_history) + heuristic(child), child))
    return None



print('h1')
print_sequence(a_star_search(problems()[2], h1))

print('h2')
print_sequence(a_star_search(problems()[2], h2))


h1
Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

h2
Steps: 10
[1, 6, 2]
[5, 7, 3]
[0, 4, 8]

[1, 6, 2]
[5, 7, 3]
[4, 0, 8]

[1, 6, 2]
[5, 0, 3]
[4, 7, 8]

[1, 0, 2]
[5, 6, 3]
[4, 7, 8]

[1, 2, 0]
[5, 6, 3]
[4, 7, 8]

[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



**d)** Compare the results obtained concerning execution time and memory space occupied
in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [7]:
import time
import tracemalloc

def solve_and_measure(problem, search_method, heuristic):
    tracemalloc.start()  # Start tracing the memory allocation
    start_time = time.time()  # Record the start time
    solution = search_method(problem, heuristic)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    current, peak = tracemalloc.get_traced_memory()  # Get the current and peak memory usage
    tracemalloc.stop()  # Stop tracing memory
    return solution, elapsed_time, peak

problems_list = problems()
heuristics = [h1, h2]
methods = [greedy_search, a_star_search]

for i, problem in enumerate(problems_list):
    print(f"Problem {i+1}")
    for heuristic in heuristics:
        for method in methods:
            solution, time_taken, memory_used = solve_and_measure(problem, method, heuristic)
            print(f"{method.__name__} with heuristic {heuristic.__name__}:")
            print(f"Time taken: {time_taken} seconds")
            print(f"Memory used: {memory_used} bytes")
            print_sequence(solution)


Problem 1
greedy_search with heuristic h1:
Time taken: 0.0020515918731689453 seconds
Memory used: 7624 bytes
Steps: 4
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

a_star_search with heuristic h1:
Time taken: 0.0015192031860351562 seconds
Memory used: 4576 bytes
Steps: 4
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

greedy_search with heuristic h2:
Time taken: 0.0020134449005126953 seconds
Memory used: 3736 bytes
Steps: 4
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

a_star_search with heuristic h2:
Time taken: 0.0019991397857666016 seconds
Memory used: 5648 bytes
Steps: 4
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

